# Tutorial 2: Interactive Decision Making - The Intervention Matrix

**Grant:** NCPTT Data-Driven Heritage Preservation

---

## 🎯 Goal: From Data to Action

In Tutorial 1, we found the problems (Factors) and the drivers (Feature Importance). 
Now, we must decide **what to do**.

**We will:**
1.  **Build an Intervention Matrix:** Prioritize actions based on data + ethics.
2.  **Play the Budget Game:** Can you save the fort with limited funds?

## 🛠️ Setup
Press `Shift + Enter` to load.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.colors import LinearSegmentedColormap

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Define Data from Tutorial 1 (Hardcoded for demo)
nscs = ['Structural Instability', 'Sill Deterioration', 'Surface Issues', 'Foundation Exposure']
interventions = ['Abstention', 'Mitigation', 'Reconstitution', 'Substitution', 'Circumvention']

# Initial Scores (Placeholder)
matrix_data = pd.DataFrame(0, index=nscs, columns=interventions)

print("✅ Setup Complete")

✅ Setup Complete


---
## 🏗️ Part 1: Build Your Matrix (Interactive)

**The Logic:**
Priority Score = (How Bad is the Problem?) × (How Good is the Solution?)

**Interactive Task:**
1.  **Enter Scores:** Type priority scores (0-100) directly into the grid below for each combination.
2.  **Think:** High score = High Priority (Red).
3.  Click **Update Heatmap** to visualize your strategy.

In [2]:
# --- INTERACTIVE MATRIX GRID ---

def create_matrix_ui():
    rows = []
    input_widgets = {} # Key: (row_name, col_name)

    # 1. Header Row
    # Empty corner + Column names
    header_items = [widgets.Label(value="", layout=widgets.Layout(width='150px'))]
    header_items += [widgets.Label(value=col, layout=widgets.Layout(width='100px', font_weight='bold')) for col in interventions]
    rows.append(widgets.HBox(header_items))

    # 2. Data Rows
    for r in nscs:
        # Row Label
        row_items = [widgets.Label(value=r, layout=widgets.Layout(width='150px', font_weight='bold'))]
        for c in interventions:
            # Input Box
            w = widgets.BoundedIntText(
                value=0, min=0, max=100, step=5, 
                layout=widgets.Layout(width='100px')
            )
            input_widgets[(r, c)] = w
            row_items.append(w)
        rows.append(widgets.HBox(row_items))

    grid = widgets.VBox(rows)
    btn_update = widgets.Button(description='Update Heatmap', button_style='success', icon='check')

    # Output area for the plot
    out_plot = widgets.Output()

    def on_click_update(b):
        # Read values from widgets into DataFrame
        for r in nscs:
            for c in interventions:
                matrix_data.loc[r, c] = input_widgets[(r, c)].value

        with out_plot:
            clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            sns.heatmap(matrix_data, annot=True, cmap='OrRd', vmin=0, vmax=100, linewidths=1, linecolor='black')
            plt.title('Intervention Priority Matrix')
            plt.show()

    btn_update.on_click(on_click_update)

    # Initial Plot
    on_click_update(None)

    return widgets.VBox([grid, btn_update, out_plot])

display(create_matrix_ui())

NameError: name 'clear_output' is not defined

---
## 💰 Part 2: The Budget Game (Interactive)

**Scenario:** You have **$10,000**.

**Interactive Task:**
1.  **Set Unit Costs:** How much does each repair cost per wall? (Enter your estimates).
2.  **Allocate Repairs:** Use the sliders to decide how many walls to fix.
3.  **Goal:** Maximize your **Total Priority Score** without going over budget!

In [ ]:
# --- BUDGET GAME WITH EDITABLE COSTS ---

budget_limit = 10000
# Priorities (Benefit per wall treated)
priorities = {'Fill': 90, 'Drip Edge': 60, 'Coat': 80, 'Sill': 70}

# 1. Widgets for Unit Costs
style_cost = {'description_width': 'initial'}
cost_widgets = {
    'Fill': widgets.IntText(value=500, description='Fill Cost ($):', style=style_cost, layout=widgets.Layout(width='200px')),
    'Drip Edge': widgets.IntText(value=800, description='Drip Cost ($):', style=style_cost, layout=widgets.Layout(width='200px')),
    'Coat': widgets.IntText(value=1500, description='Coat Cost ($):', style=style_cost, layout=widgets.Layout(width='200px')),
    'Sill': widgets.IntText(value=1200, description='Sill Cost ($):', style=style_cost, layout=widgets.Layout(width='200px'))
}

# 2. Widgets for Quantities
qty_widgets = {
    'Fill': widgets.IntSlider(min=0, max=20, description='Qty:', layout=widgets.Layout(width='300px')),
    'Drip Edge': widgets.IntSlider(min=0, max=20, description='Qty:', layout=widgets.Layout(width='300px')),
    'Coat': widgets.IntSlider(min=0, max=20, description='Qty:', layout=widgets.Layout(width='300px')),
    'Sill': widgets.IntSlider(min=0, max=20, description='Qty:', layout=widgets.Layout(width='300px'))
}

def update_budget_game(c_fill, c_drip, c_coat, c_sill, q_fill, q_drip, q_coat, q_sill):
    # Gather inputs
    current_costs = {'Fill': c_fill, 'Drip Edge': c_drip, 'Coat': c_coat, 'Sill': c_sill}
    qtys = {'Fill': q_fill, 'Drip Edge': q_drip, 'Coat': q_coat, 'Sill': q_sill}

    # Calculate
    total_cost = sum(current_costs[k] * qtys[k] for k in current_costs)
    total_score = sum(priorities[k] * qtys[k] for k in priorities)

    remaining = budget_limit - total_cost
    color = 'green' if remaining >= 0 else 'red'

    # Text Output
    print(f"TOTAL COST: ${total_cost:,}")
    print(f"REMAINING:  ${remaining:,}")
    print(f"PRIORITY SCORE ACHIEVED: {total_score}")

    if remaining < 0:
        print("❌ OVER BUDGET! Reduce quantities or lower costs.")

    # Bar Chart
    plt.figure(figsize=(10, 2))
    plt.barh(['Budget Used'], [total_cost], color=color)
    plt.axvline(budget_limit, color='black', linestyle='--', label='Limit ($10k)')
    plt.xlim(0, max(15000, total_cost + 1000))
    plt.legend()
    plt.show()

# Layout Construction
rows = []
labels = {'Fill': 'Fill Foundation', 'Drip Edge': 'Drip Edge', 'Coat': 'Shelter Coat', 'Sill': 'Rebuild Sill'}

for key in ['Fill', 'Drip Edge', 'Coat', 'Sill']:
    # Row: [Label] [Cost Input] [Qty Slider]
    row = widgets.HBox([
        widgets.Label(value=labels[key], layout=widgets.Layout(width='120px', font_weight='bold')),
        cost_widgets[key],
        qty_widgets[key]
    ])
    rows.append(row)

ui_budget = widgets.VBox(rows)

# Link widgets to function
out_budget = widgets.interactive_output(update_budget_game, {
    'c_fill': cost_widgets['Fill'], 'c_drip': cost_widgets['Drip Edge'],
    'c_coat': cost_widgets['Coat'], 'c_sill': cost_widgets['Sill'],
    'q_fill': qty_widgets['Fill'], 'q_drip': qty_widgets['Drip Edge'],
    'q_coat': qty_widgets['Coat'], 'q_sill': qty_widgets['Sill']
})

display(ui_budget, out_budget)

---
## 🙏 Acknowledgements

This material was developed under a grant from the **National Center for Preservation Technology and Training (NCPTT)**.

*Data-Driven Heritage Preservation: Leveraging Machine Learning for Informed Adobe Conservation Strategies*